In [5]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
#from requests_testadapter import Resp
import requests
#from requests_file import FileAdapter
import os.path


In [10]:
#limpiamos el DF en memoria para iniciar el proceso.
df = pd.DataFrame()
df_reviews = pd.DataFrame()

#ruta de donde vamos a tomar los htmls.
ruta = r"C:/Users/Joaquin/Documents/GitHub/desafio-final/Data/Prueba/c/"

#recorremos todos los archivos en la ruta. 
for filename in os.listdir(ruta):
        file =  open(ruta+filename, "r", encoding='utf-8') #abrimos el archivo

        bsObj = BeautifulSoup(file, 'html.parser') #generamos el objeto beautiful soup
        bsObj = bsObj.encode('utf-8')
        #print(bsObj.original_encoding)
    

#TOMA DE CONTENIDO - ETIQUETAS HTML
                       
        aclass = bsObj.findAll("a", {"class":"display-extended-links"}) #obtenemos barrio y tipo comida de esta var
        
        tipo_comida_pre = bsObj.findAll("a", {"class":"display-extended-links"}) 
        tipo_comida = []                      
        if len(tipo_comida_pre) - 2 >= 0:
            for i in range( 0, len(tipo_comida_pre) - 2 ):
                tipo_comida.append(tipo_comida_pre[i].get_text())     
        
        
        actividades_pre = bsObj.findAll("h4",{"class":"tile-text-sm description"})
        actividades= []
        for i in range( 0, len(actividades_pre)):
            actividades.append(actividades_pre[i].get_text())
        
        
        extras_pre = bsObj.findAll("h4",{"class":"title-text-sm description"})
        extras = []
        for i in range( 0, len(extras_pre)):
            extras.append(extras_pre[i].get_text())
        
        barrio = []
        if len(aclass) -2 >= 0 :
            barrio = bsObj.findAll("a", {"class":"display-extended-links"})[len(aclass)-2].text 

        nombre_lugar = filename[0:len(filename)-5] 

        
        price_range = []
        #price_range = bsObj.find("span", {"class":"price-range"}) - version vieja borrar
        price_range_b = bsObj.find("span", {"class":"price-range"})
        if price_range_b != None:
            price_range = price_range_b['class'][1].replace("price-range-","")
 
    
        reviews = bsObj.findAll("p",{"class":"activity-text"})
        reviews_date = bsObj.findAll("div",{"class":"activity-date"})
        print(reviews)
        
        #en proceso
        puntaje_review = []      
        puntaje_review = bsObj.findAll("div",{"class":"activity-ratings"})
#        puntaje_review_b = puntaje_review[0].findAll("span")
#         review_comida_b = puntaje_review_b[2].get_text().strip()
#         review_servicio_b = puntaje_review_b[5].text
#         review_ambiente_b = puntaje_review_b[8].text
#         review_comida = " ".join(review_comida_b.split())
#         review_servicio = " ".joinreview_servicio_b.split())
#         review_ambiente = " ".join(review_ambiente_b.split())
        
        
        
        #GUARDADO DE DATOS CON APPEND MEDIANTE ITERACIONES
        
        serie = pd.Series([nombre_lugar, tipo_comida, barrio, price_range, actividades, extras], index=['nombre_lugar', 'tipo_comida', 'barrio', 'price_range', 'actividades', 'extras'])
        df = df.append(serie, ignore_index=True)
        
        serie_reviews = pd.Series([nombre_lugar, reviews, reviews_date, puntaje_review]) #index=['nombre_lugar', 'reviews'])
        df_reviews = df_reviews.append(serie_reviews, ignore_index=True)

        

export_csv = df.to_csv ('df_scrapping_nuevo.csv', index = None, header=True) 


AttributeError: 'bytes' object has no attribute 'findAll'

In [11]:
df_reviews

,0,1,2,3
0,N53 Fernet Bar Palermo -,"[<p class=""activity-text"">Los platos no son ab...","[<div class=""activity-date"">06/11/2010</div>, ...","[<div class=""activity-ratings""> <div> <span>Co..."


In [12]:
#tomamos df_reviews generado en el paso anterior e iteramos por la lista de reviews y sus fechas y las guardamos como registros
#independientes en un nuevo dataframe.

df_reviews_shaped = pd.DataFrame()
resto_counter = 0
for resto in df_reviews[0].values:
    counter2 = 0
    for item in df_reviews.iloc[resto_counter][1]:
        review = df_reviews.iloc[resto_counter][1][counter2].text
        
        print(review)
        
        fecha = df_reviews.iloc[resto_counter][2][counter2].text
        
        #iteramos dentro de puntaje y la descomponemos en columna comida, servicio y ambiente.
        #agregamos puntaje al DF reviews, por si queremos controlar la info, despues lo dropeamos.
        try:
            puntaje = df_reviews.iloc[resto_counter][3][counter2]
            puntaje = puntaje.findAll("span")

            comida = puntaje[2].get_text()
            comida = comida.replace('\\t','').replace('\\n','').strip()

            servicio = puntaje[5].get_text()
            servicio = servicio.replace('\\t','').replace('\\n','').strip()

            ambiente = puntaje[8].get_text()
            ambiente = ambiente.replace('\\t','').replace('\\n','').strip()
            
        except IndexError:
            puntaje = None
            comida = None
            servicio = None
            ambiente = None
        
        
        
        serie_reviews_2 = pd.Series([resto, review, fecha, puntaje, comida, servicio, ambiente], index=['resto', 'review', 'fecha', 'puntaje', 'comida', 'servicio', 'ambiente'])
        df_reviews_shaped = df_reviews_shaped.append(serie_reviews_2, ignore_index=True)
        counter2=counter2 + 1
    resto_counter = resto_counter + 1
    
    df_reviews_shaped = df_reviews_shaped[['resto','fecha','review','comida', 'servicio', 'ambiente']]
        
#export_csv = df_reviews.to_csv (r'C:/Users/QQ-Haum/Documents/desafio-final/df_reviewss.csv', index = None, header=True) 
#export_csv = df_reviewss.to_csv (r'C:/Users/QQ-Haum/Documents/desafio-final/df_reviewss.csv', index = None, header=True) 
export_csv = df_reviews_shaped.to_csv ('out.csv', index = None, header=True, encoding='utf-8') 



Los platos no son abundantes ni tan bien logrados, La atención más bien floja
Riquísimo y muy agradable.Caí un sábado al mediodía temprano y y me senté en una preciosa mesita al sol, escuché buen jazz, pude leer sin que nadie me molestara. La atención de los mozos y del encargado fue excelente y los precios acorde a lo que consumí y a la zona. Lo recomiendo: el ambiente es súper relajado y no es ruidoso aún cdo se llena de gente.
Excelente el salmón, lo recomiendo. Una porción muy generosa y sabrosa. Además, con papas y ensalada.
La atención muy buena y la ambientación muy linda. Sin embargo, los platos escasos... La verdad nos quedamos con hambre y por esta razón es que nos pareció caro. No tienen cerveza de litro, sino porrones y también nos parecieron caros.
Un exquisito bistro en Palermo. Menú reducido pero eficiente. Atención cálida. Imperdible la chocotorta. Volveremos
comimos muy exquisito de noche. Tienen wi.fi, eso es bueno. Volví despues al mediodía con amigas, el lugar es ag